In [146]:
from operator import methodcaller
import string
import re
from collections import Counter, defaultdict
import numpy as np
from itertools import islice
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
import torch
import torch.optim as optim
import torch.nn as nn

In [147]:
def expand_contradictions(text):

    contraction_mapping = {
        "won't": "will not",
        "can't": "can not",
        "n't": " not",
        "'re": " are",
        "'s": " is",
        "'d": " would",
        "'ll": " will",
        "'ve": " have",
        "'m": " am"
    }

    pattern = re.compile(r"\b(?:" + "|".join(re.escape(contraction) for contraction in contraction_mapping.keys()) + r")\b")
    text = pattern.sub(lambda x: contraction_mapping[x.group()], text)
    
    return text

def remove_digits_and_words_digits(text):
    # Define a regular expression pattern to match words containing digits
    pattern = r'\b\w*\d\w*\b'
    text_without_words_with_digits = re.sub(pattern, '', text)

    return text_without_words_with_digits

def remove_stop_words(text):
    stop_words = ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll",
                  "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself',
                  'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 
                  'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those',
                  'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an',
                  'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against',
                  'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on',
                  'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any',
                  'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than',
                  'too', 'very', 's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', 've',
                  'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn', "hadn't", 'hasn', "hasn't", 'haven',
                  "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn', "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't",
                  'wasn', "wasn't", 'weren', "weren't", 'won', "won't", 'wouldn', "wouldn't"]
    text_clean = []
    for word in text:
        if word not in stop_words:
            text_clean.append(word)
    return text_clean


def tokenize(text, split=True):
    # TODO customize to your needs
    text = text.translate(str.maketrans({key: " {0} ".format(key) for key in string.punctuation}))
    # re.sub('[^a-zA-Z]', '', dataset['Text'][i])

    # Text preprocessing techniques:
    # 1) Lowercase
    text = text.lower()

    # 2) Expand Contradictions
    text = expand_contradictions(text)

    # 3) Remove punctuations
    text = re.sub('[%s]' % re.escape(string.punctuation), '' , text)

    # 4) Remove digits and words with digits
    text = remove_digits_and_words_digits(text)
    if split:
        text = text.split()

    # # 5) Remove Stop Words
    # if stop_words:
    # text = remove_stop_words(text)

    return text

class Features:

    def __init__(self, data_file):
        with open(data_file) as file:
            data = file.read().splitlines()

        data_split = map(methodcaller("rsplit", "\t", 1), data)
        texts, self.labels = map(list, zip(*data_split))

        self.tokenized_text = [tokenize(text) for text in texts]

        self.labelset = list(set(self.labels))

    @classmethod 
    def get_features(cls, tokenized, model):
        # TODO: implement this method by implementing different classes for different features 
        # Hint: try simple general lexical features first before moving to more resource intensive or dataset specific features 
        pass


class Features_FeedForward(Features):

    def __init__(self, input_file, embedding_file):
        super(Features_FeedForward, self).__init__(input_file)
        self.embedding_matrix = self.read_embedding_file(embedding_file) # Need to save EmbeddingMatrix values for inference

    def adjust_max_seq_length(self, tokenized_text, max_seq_length):
        """Adjust size of data input to the max sequence length
        :param tokenized_text: data input
        :param max_seq_length: the max sequence length
        :return list: truncated sentences
        """
        new_tokenized_text = []
        for sentence in tokenized_text:
            new_tokenized_text.append(sentence[:max_seq_length])
        return new_tokenized_text

        
    def read_embedding_file(self, embedding_file):
        '''Read embedding file

        :param embedding_file (str):
        :return: dict: embedding matrix
        '''

        embedding_matrix = dict()
        try: 
            with open(embedding_file, "r") as file:
                for line in file:
                    values = line.strip().split()
                    word = values[0]
                    word_embedding = np.array([float(emb) for emb in values[1:]])
                    embedding_matrix[word] = word_embedding
            return embedding_matrix
        except OSError as e:
            print("Embedding file " + embedding_file + " is not available, please input the right parth to the file.")

    def read_inference_file(self, input_file):
        """Read inference file that is in the form: <text> i.e. a line
        of text that does not contain a tab.
        """
        with open(input_file) as file:
            data = file.read().splitlines()

        texts = data

        tokenized_text = [tokenize(text) for text in texts]
        return tokenized_text
    
    def get_features(self, tokenized_sentence):
        """Convert sentence to word embeeding values.
        :param tokenized_sentence
        :return feature weights
        """
        sentence_embedding = []
        
        for word in tokenized_sentence:
            # get embedding of word if exists
            try:
                word_emb = self.embedding_matrix[word]
            except: # read UNK token embedding 
                word_emb = self.embedding_matrix["UNK"]
            sentence_embedding.append(word_emb)
        
        return sentence_embedding

In [148]:
class NeuralNetworkTorch(nn.Module):
    def __init__(self, input_dim, hidden_units, n_labels):
        super(NeuralNetworkTorch, self).__init__()
        self.n_hidden_layers = len(hidden_units)
        self.activation = nn.ReLU()
        # List of domensions
        self.list_of_sizes = [input_dim] + hidden_units + [n_labels]

        self.weights = nn.ModuleList([nn.Linear(self.list_of_sizes[i], self.list_of_sizes[i+1]) for i in range(self.n_hidden_layers)])

        self.output_layer = nn.Linear(hidden_units[-1], n_labels)
        # Last actvatio is a Softmax
        self.softmax = nn.Softmax(dim=1) # Apply softmax along the output dimension
        
        # self.weights_1 = nn.Linear(input_dim, hidden_units)
        # self.relu = nn.ReLU()
        # self.weights_2 = nn.Linear(hidden_units, n_labels)
        # self.softmax = nn.Softmax(dim=1)  # Apply softmax along the output dimension

    def forward(self, x):
        # ---------------- Input-to-Hidden Layer --------------- #

        for i in range(self.n_hidden_layers):
            x = self.weights[i](x)
            x = self.activation(x)
        # ---------------- Hidden-to-Output Layer --------------- #

        x = self.output_layer(x)
        x = self.softmax(x)
        return x

In [241]:
from model import Model

class NeuralModel_Torch(Model):
    def __init__(self, embeddingfile, max_seq_length, hidden_units, minibatch_size, learning_rate, epochs, hidden_units_other_layers=[],adam=False, average_emb_sentence=False): 
        self.embeddingfile = embeddingfile
        self.embedding_dim = None
        self.max_seq_length = max_seq_length
        self.hidden_units = [hidden_units] +  hidden_units_other_layers if len(hidden_units_other_layers) > 0 else [hidden_units]
        self.average_emb_sentence = average_emb_sentence
        # Layers
        self.model_torch = None
        self.Y_to_categorical = None
        self.criterion = nn.CrossEntropyLoss()
        self.minibatch_size = minibatch_size
        self.epochs = epochs
        self.features_ff_class = None
        self.learning_rate = learning_rate
        self.adam = adam
        self.loss = {}
        
    def convert_to_embeddings(self, sentence, average_emb_sentence=False):
        '''Convert sentence to embeddings.

        :param average_sentence:  Compute the element-wise average of a list of sentence embedding
        '''
        emb = self.features_ff_class.get_features(sentence)
        if not average_emb_sentence:
            if emb: # if there is a word
                emb_concat = np.concatenate(emb, axis=0)
            else:
                emb_concat = []
            # If you need padding words (i.e., your input is too short), use a vector of zeroes
            if len(emb) < self.max_seq_length:
                # Missing words
                words_missing = self.max_seq_length - len(emb)
                # print(words_missing)
                emb_concat = np.pad(emb_concat, (0, words_missing*self.embedding_dim), 'constant')
        else:
            # Compute average of the sentence
            if len(emb) > 0:
                stacked_arrays = np.vstack(emb)
                emb_concat = np.mean(stacked_arrays, axis=0)
            else:
                emb_concat = np.zeros(self.embedding_dim)

        return emb_concat

    
    def train(self, input_file, verbose=False):

        # Read dataset and create vocabulary
        features_ff_class = Features_FeedForward(input_file, self.embeddingfile)
        self.features_ff_class = features_ff_class
        num_labels = len(features_ff_class.labelset)

        # Convert Y from categorical to integers values
        Y_mapping = {label: index for index, label in enumerate(np.unique(features_ff_class.labels))}
        self.Y_to_categorical = {index: label for label, index in Y_mapping.items()} # dictionary to convert back y's to categorical
        Y = [Y_mapping[y] for y in features_ff_class.labels]
        Y = np.array(Y)
        # Convert to OneHot for computing Loss
        # Y_onehot = self.OneHot(Y, num_labels)

        # Get embedding dim
        self.embedding_dim = list(features_ff_class.embedding_matrix.values())[0].shape[0]

        # Number of sentences
        sample_size = len(features_ff_class.tokenized_text)

        # X_train: shape: 50f or 300f-dim × features (u)
        if not self.average_emb_sentence:
            n_inputs = self.max_seq_length*self.embedding_dim # number of features
        else:
            n_inputs = self.embedding_dim
        X_train = np.zeros((sample_size, n_inputs))

        # Truncate input to the max sequence length
        trunc_tokenized_text = features_ff_class.adjust_max_seq_length(
            features_ff_class.tokenized_text,
            self.max_seq_length
        )

        # Convert to embeddings with zero-padding
        for i, sentence in enumerate(trunc_tokenized_text):
            sentence_emb = self.convert_to_embeddings(sentence, average_emb_sentence=self.average_emb_sentence)
            X_train[i] = sentence_emb

        minibatch_size = self.minibatch_size

        # Initialize Torch Model
        self.model_torch = NeuralNetworkTorch(n_inputs, self.hidden_units, num_labels)
        # Optimzer
        if self.adam:  
            optimizer = optim.Adam(self.model_torch.parameters(), lr=self.learning_rate)
        else:
            optimizer = optim.SGD(self.model_torch.parameters(), lr=self.learning_rate)

        #################
        # Torch Tensors #
        #################
        # Permutate the dataset to increase randomness
        np.random.seed(0)
        permutation = np.random.permutation(sample_size)
        # X_train[n_documents, n_features]
        X_permutation = X_train[permutation]
        Y_permutation = Y[permutation]


        # Torch Tensors
        X_permutation = torch.tensor(X_permutation, dtype=torch.float32)
        Y_permutation = torch.tensor(Y_permutation)

        for i in range(self.epochs):
            # Mini-batch_size Implementation
            mini_batch_loss = []
            for j in range(0, sample_size, minibatch_size):
                X_mini_batch = X_permutation[j:j+minibatch_size]
                y_mini_batch = Y_permutation[j:j+minibatch_size]

                ##########################################################
                # ---------------------FORWARD PASS--------------------- #
                ##########################################################
                optimizer.zero_grad()
                outputs = self.model_torch.forward(X_mini_batch)

                loss = self.criterion(outputs, y_mini_batch)

                ##########################################################
                # -------------------BACKWARD PASS---------------------- #
                ##########################################################
   
                loss.backward()
                optimizer.step()

                mini_batch_loss.append(loss.item())

            
            self.loss[i] = np.mean(mini_batch_loss)
            if verbose:
                print(f"Epoch: {i+1} - Loss: {self.loss[i]}")

    def classify(self, input_file):
        # Read Input File
        tokenized_text = self.features_ff_class.read_inference_file(input_file)

        # Truncate input to the max sequence length
        trunc_tokenized_text = self.features_ff_class.adjust_max_seq_length(
            tokenized_text,
            self.max_seq_length
        )
        X_test = []
        # Convert to embeddings with zero padding
        for i, sentence in enumerate(trunc_tokenized_text):
            sentence_emb = self.convert_to_embeddings(sentence, average_emb_sentence=self.average_emb_sentence)
            X_test.append(sentence_emb)
        X_test = np.vstack(X_test)

        # Convert to tensor
        X_test = torch.tensor(X_test, dtype=torch.float32)

        # Make Prediction
        preds_label = []
        with torch.no_grad():
            predicted = self.model_torch(X_test)
            _, y_test = torch.max(predicted, 1)
            for y in y_test:
                tmp = self.Y_to_categorical[y.item()] # Convert to original class
                preds_label.append(tmp)
        
        return preds_label

In [642]:
############
# Products #
############

# train_file = "datasets/products/train.txt"
# emb_file = "glove.6B.50d.txt"
# pred_file = "datasets/products/val.test"
# pred_true_labels = "datasets/products/val.txt"
# model_file_name = "torch.products.model"
# loss_file = "datasets/products/loss.txt"

# nn_model = NeuralModel_Torch(emb_file, max_seq_length=10, hidden_units=5, minibatch_size=32, learning_rate=0.1, epochs=100)
# nn_model = NeuralModel_Torch(emb_file, max_seq_length=10, hidden_units=5, minibatch_size=32, learning_rate=0.1, epochs=100)

# 1 Layers
# nn_model = NeuralModel_Torch(emb_file, max_seq_length=15, hidden_units=10, minibatch_size=256, learning_rate=0.2, epochs=500)#, tfidf=True, max_features=1000, threshold=2) # 66.33%
##BEST MODEL###
# nn_model = NeuralModel_Torch(emb_file, max_seq_length=60, hidden_units=20, minibatch_size=256, learning_rate=0.2, epochs=300, average_emb_sentence=True)
# nn_model = NeuralModel_Torch(emb_file, max_seq_length=40, hidden_units=10, hidden_units_other_layers=[5, 5], minibatch_size=32, learning_rate=0.001, epochs=200, average_emb_sentence=True) 

# nn_model = NeuralModel_Torch(emb_file, max_seq_length=40, hidden_units=10, hidden_units_other_layers=[10], minibatch_size=32, learning_rate=0.02, epochs=200, average_emb_sentence=True) 

# 2 Layers

# nn_model = NeuralModel_Torch(emb_file, max_seq_length=40, hidden_units=4, hidden_units_other_layers=[4], minibatch_size=32, learning_rate=0.02, epochs=200)

########
# 4dim #
########

# train_file = "datasets/4dim/train.txt"
# emb_file = "glove.6B.50d.txt"
# pred_file = "datasets/4dim/val.test"
# pred_true_labels = "datasets/4dim/val.txt"
# model_file_name = "torch.4dim.model"
# loss_file = "datasets/4dim/loss.txt"
# nn_model = NeuralModel_Torch(emb_file, max_seq_length=10, hidden_units=5, minibatch_size=32, learning_rate=0.1, epochs=100) # 39
# nn_model = NeuralModel_Torch(emb_file, max_seq_length=40, hidden_units=5, minibatch_size=32, learning_rate=0.001, epochs=300, adam=True) # 43%
# nn_model = NeuralModel_Torch(emb_file, max_seq_length=40, hidden_units=10, minibatch_size=32, learning_rate=0.01, epochs=300, adam=True) # 45%
# nn_model = NeuralModel_Torch(emb_file, max_seq_length=40, hidden_units=10, minibatch_size=32, learning_rate=0.015, epochs=300, adam=True) # 45.51%
# nn_model = NeuralModel_Torch(emb_file, max_seq_length=40, hidden_units=10, minibatch_size=32, learning_rate=0.005, epochs=500, adam=True) # 45.51%


# nn_model = NeuralModel(emb_file, max_seq_length=40, hidden_units=100, minibatch_size=256, learning_rate=0.0015, epochs=500, momentum=0.9, tfidf=True, threshold=0, max_features=10000) # Accuracy: 46.47%
# nn_model = NeuralModel_Torch(emb_file, max_seq_length=40, hidden_units=100, minibatch_size=256, learning_rate=0.0015, epochs=500, adam=False) # Accuracy: 33.33%

# 1 Layer
# nn_model = NeuralModel_Torch(emb_file, max_seq_length=40, hidden_units=100, minibatch_size=32, learning_rate=0.002, epochs=500)

# nn_model = NeuralModel_Torch(emb_file, max_seq_length=40, hidden_units=100, minibatch_size=32, learning_rate=0.0001, epochs=500, average_emb_sentence=True, adam=True)
##BEST MODEL###
# nn_model = NeuralModel_Torch(emb_file, max_seq_length=100, hidden_units=30, minibatch_size=32, learning_rate=0.001, epochs=500, average_emb_sentence=True, adam=True)

# nn_model = NeuralModel_Torch(emb_file, max_seq_length=40, hidden_units=80, hidden_units_other_layers=[10, 10], minibatch_size=64, learning_rate=0.0001, epochs=500, average_emb_sentence=True) # Accuracy: 65.06%

# nn_model = NeuralModel_Torch(emb_file, max_seq_length=40, hidden_units=80, minibatch_size=32, learning_rate=0.0015, epochs=500, average_emb_sentence=True, adam=False)

# max_seq_length=40, hidden_units=80, minibatch_size=64, learning_rate=0.0015, epochs=500, momentum=0.95, tfidf=True
# 2 Layer
# nn_model = NeuralModel_Torch(emb_file, max_seq_length=40, hidden_units=50, hidden_units_other_layers=[50], minibatch_size=64, learning_rate=0.001, epochs=200)
#############
# questions #
#############

# train_file = "datasets/questions/train.txt"
# emb_file = "ufvytar.100d.txt"
# pred_file = "datasets/questions/val.test"
# pred_true_labels = "datasets/questions/val.txt"
# model_file_name = "torch.questions.model"
# loss_file = "datasets/questions/loss.txt"
# # nn_model = NeuralModel_Torch(emb_file, max_seq_length=10, hidden_units=5, minibatch_size=32, learning_rate=0.1, epochs=100)


# # Validation
# # 1 Layer
##BEST MODEL###
# nn_model = NeuralModel_Torch(emb_file, max_seq_length=10, hidden_units=5, minibatch_size=32, learning_rate=0.1, epochs=200)

# 2 Layer
# nn_model = NeuralModel_Torch(emb_file, max_seq_length=50, hidden_units=5, hidden_units_other_layers=[5], minibatch_size=128, learning_rate=0.025, epochs=400)

# nn_model = NeuralModel_Torch(emb_file, max_seq_length=40, hidden_units=5, hidden_units_other_layers=[5, 5], minibatch_size=128, learning_rate=0.2, epochs=400, average_emb_sentence=True) # Accuracy: 61.86%
#########
# odiya #
#########

train_file = "datasets/odiya/train.txt"
emb_file = "fasttext.wiki.300d.vec"
pred_file = "datasets/odiya/val.test"
pred_true_labels = "datasets/odiya/val.txt"
model_file_name = "torch.odia.model"
loss_file = "datasets/odiya/loss.txt"
# # # nn_model = NeuralModel_Torch(emb_file, max_seq_length=10, hidden_units=5, minibatch_size=32, learning_rate=0.1, epochs=100)

# # # 1 Layer
# # # nn_model = NeuralModel_Torch(emb_file, max_seq_length=10, hidden_units=5, minibatch_size=32, learning_rate=0.1, epochs=100)

# # # 2 Layer
# ##BEST MODEL###
# nn_model = NeuralModel_Torch(emb_file, max_seq_length=30, hidden_units=5, hidden_units_other_layers=[5], minibatch_size=128, learning_rate=0.01, epochs=200) 

nn_model = NeuralModel_Torch(emb_file, max_seq_length=30, hidden_units=5, hidden_units_other_layers=[5, 5], minibatch_size=128, learning_rate=0.01, epochs=200) 

# nn_model = NeuralModel(emb_file, max_seq_length=10, hidden_units=10, minibatch_size=64, learning_rate=0.35, epochs=100)
# nn_model = NeuralModel(emb_file, max_seq_length=20, hidden_units=10, minibatch_size=64, learning_rate=0.1, epochs=1000)
# nn_model = NeuralModel(emb_file, max_seq_length=10, hidden_units=5, minibatch_size=32, learning_rate=0.1, epochs=100)


In [643]:
nn_model.train(train_file, verbose=True)

Epoch: 1 - Loss: 1.092742357755962
Epoch: 2 - Loss: 1.0883695351450067
Epoch: 3 - Loss: 1.0845312545174046
Epoch: 4 - Loss: 1.0808350600694354
Epoch: 5 - Loss: 1.077128681383635
Epoch: 6 - Loss: 1.0737557235517
Epoch: 7 - Loss: 1.07076062905161
Epoch: 8 - Loss: 1.0681435886182282
Epoch: 9 - Loss: 1.065899618048417
Epoch: 10 - Loss: 1.0638974139564916
Epoch: 11 - Loss: 1.0620200922614649
Epoch: 12 - Loss: 1.0601877789748342
Epoch: 13 - Loss: 1.0582967582501863
Epoch: 14 - Loss: 1.05633418810995
Epoch: 15 - Loss: 1.0542912633795487
Epoch: 16 - Loss: 1.0521641066199854
Epoch: 17 - Loss: 1.0498961022025661
Epoch: 18 - Loss: 1.0473855972290038
Epoch: 19 - Loss: 1.044520233806811
Epoch: 20 - Loss: 1.0411638535951313
Epoch: 21 - Loss: 1.037117970617194
Epoch: 22 - Loss: 1.0320707333715338
Epoch: 23 - Loss: 1.025720475221935
Epoch: 24 - Loss: 1.0180110730622944
Epoch: 25 - Loss: 1.00897431185371
Epoch: 26 - Loss: 0.9987533418755782
Epoch: 27 - Loss: 0.9876566190468637
Epoch: 28 - Loss: 0.97613

In [644]:
nn_model.save_model(model_file_name)

In [645]:
model = NeuralModel_Torch.load_model(model_file_name)

In [646]:
preds= model.classify(pred_file + ".txt")

In [647]:
## Save the predictions: one label prediction per line
with open(pred_file + ".pred.txt", "w") as file:
    for pred in preds:
        file.write(pred+"\n")

# Evaluation

In [648]:
import pandas as pd

In [649]:
# products
# pred_file = "datasets/products/val.test"
# pred_true_labels = "datasets/products/val.txt"

# 4dim

# pred_file = "datasets/4dim/val.test"
# pred_true_labels = "datasets/4dim/val.txt"


# questions
# pred_file = "datasets/questions/val.test"
# pred_true_labels = "datasets/questions/val.txt"

# odiya

# pred_file = "datasets/odiya/val.test"
# pred_true_labels = "datasets/odiya/val.txt"

In [650]:
true_dataset = pd.read_csv(pred_true_labels, sep='\t', header=None, names=['text', 'true_label'])
pred_dataset = pd.read_csv(pred_file + ".pred.txt", sep='\t', header=None, names=['pred'])

In [651]:
true_dataset.true_label.value_counts()

sports           1345
business          937
entertainment     758
Name: true_label, dtype: int64

In [652]:
pred_dataset.value_counts()

pred         
sports           1437
business          900
entertainment     703
dtype: int64

In [653]:
# Check if the columns have the same name; adjust as needed
column_name = 'true_label'  # Change to the actual column name
pred_column_name = 'pred'  # Change to the actual predicted column name

# Merge the two DataFrames on a common index or key if available
merged_df = true_dataset.merge(pred_dataset, left_index=True, right_index=True)

# Calculate the accuracy by comparing the two columns
accuracy = (merged_df[column_name] == merged_df[pred_column_name]).mean()

# Print the accuracy as a percentage
print(f'Accuracy: {accuracy * 100:.2f}%')

Accuracy: 81.51%
